# Make data

In [1]:
import pandas as pd
import elmada

COUNTRIES = [
        'AT', 'BE', 'CZ', 'DE', 'DK', 'ES', 'FI', 'FR', 'GB', 'GR', 'HU', 'IE',
        'IT', 'LT', 'NL', 'PL', 'PT', 'RO', 'RS', 'SI'
    ]
FREQS = ["60min", "15min"]
YEARS = range(2017, 2021)

In [2]:
def make_data(year, country, freq):
    df = elmada.get_emissions(year=year, country=country, freq="60min", method="_PWL")
    df.index.name = "T"
    if freq == "15min":
        df = elmada.helper.upsample(df, year=year)
    df.to_csv(f"data/{freq}/CEFs_{year}_{country}.csv")

def check_length(year, country, freq):
    df = pd.read_csv(f"data/{freq}/CEFs_{year}_{country}.csv")
    steps_per_day = 96 if freq == "15min" else 24
    days_per_year = 366 if year%4==0 else 365
    assert len(df) == days_per_year * steps_per_day

In [8]:
# EXPENSIVE!

for freq in FREQS:
    for year in YEARS:
        for country in COUNTRIES:
            make_data(year=year, country=country, freq=freq)

In [9]:
for freq in FREQS:
    for year in YEARS:
        for country in COUNTRIES:
            check_length(year=year, country=country, freq=freq)

# Make time zones for readme

In [7]:
import pytz
import datetime

print("| Country | UTC offset | Time zone |")
print("| ------- | ---------- | -------- |")
d = dict()
for cy in elmada.mappings.COUNTRIES_FOR_ANALYSIS:
    tz = pytz.timezone(elmada.from_entsoe.get_timezone(cy))
    delta_seconds = tz.utcoffset(datetime.datetime(2019,1,1))
    delta_hours = delta_seconds.seconds // 3600
    print(f"| {cy} | UTC+{delta_hours:.0f}:00 | {tz} |")

| Country | UTC offset | Time zone |
| ------- | ---------- | -------- |
| AT | UTC+1:00 | Europe/Vienna |
| BE | UTC+1:00 | Europe/Brussels |
| CZ | UTC+1:00 | Europe/Prague |
| DE | UTC+1:00 | Europe/Berlin |
| DK | UTC+1:00 | Europe/Copenhagen |
| ES | UTC+1:00 | Europe/Madrid |
| FI | UTC+2:00 | Europe/Helsinki |
| FR | UTC+1:00 | Europe/Paris |
| GB | UTC+0:00 | Europe/London |
| GR | UTC+2:00 | Europe/Athens |
| HU | UTC+1:00 | Europe/Budapest |
| IE | UTC+0:00 | Europe/Dublin |
| IT | UTC+1:00 | Europe/Rome |
| LT | UTC+2:00 | Europe/Vilnius |
| NL | UTC+1:00 | Europe/Amsterdam |
| PL | UTC+1:00 | Europe/Warsaw |
| PT | UTC+0:00 | Europe/Lisbon |
| RO | UTC+2:00 | Europe/Bucharest |
| RS | UTC+1:00 | Europe/Belgrade |
| SI | UTC+1:00 | Europe/Ljubljana |
